# Spotify Recommendation System - Experimentation Notebook

This notebook provides a complete framework for training, evaluating, and visualizing a baseline collaborative filtering model for song recommendations.

## 1. Setup

Install and import necessary libraries. We add `matplotlib` for plotting.

In [8]:
!pip install pandas scikit-learn tqdm matplotlib


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [9]:
import pandas as pd
import numpy as np
import json
import os
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import matplotlib.pyplot as plt

## 2. Experiment Tracking Setup

We'll create a list to store the results of each experiment. This will allow us to easily compare runs and plot the outcomes.

In [10]:
experiment_results = []

## 3. Core Functions (Data Loading, Evaluation)

These are the helper functions for loading data and calculating our evaluation metrics.

In [ ]:
# Loads one or more JSON slice files from the dataset.
# 'path' is the absolute path to the 'data' folder of the MPD.
# 'slice_nums' is a list of slice indices (e.g., [0] → playlists 0–999, [0,1] → 0–1999).
def load_playlist_slice(path, slice_nums=[0]):
    all_playlists = []
    for slice_num in slice_nums:
        filename = f'mpd.slice.{slice_num*1000}-{(slice_num+1)*1000-1}.json'
        try:
            with open(os.path.join(path, filename)) as f:
                data = json.load(f)
                all_playlists.extend(data['playlists'])
        except FileNotFoundError:
            print(f"Warning: Slice file not found at {os.path.join(path, filename)}. Skipping.")
    return all_playlists


# Precision@k:
# "Out of the top 'k' recommendations, how many were actually relevant?"
# 'recommendations' → items your model suggested.
# 'holdout_items' → actual items the user liked.
def precision_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / k


# Recall@k:
# "Of all the relevant items, what proportion did we find?"
def recall_at_k(k, recommendations, holdout_items):
    recs_at_k = recommendations[:k]
    hits = len(set(recs_at_k) & set(holdout_items))
    return hits / len(holdout_items) if len(holdout_items) > 0 else 0


## 4. Experiment Runner

This is the main function that encapsulates the entire process: data loading, preprocessing, training, and evaluation. It takes your parameters, runs the experiment, and stores the results.

In [16]:
def run_experiment(slice_nums, n_components, k=10): #k here generates 10 songs
    """Runs a full experiment cycle with given parameters and stores the results."""
    
    # --- 1. Data Loading & Preprocessing ---
    print(f"Loading {len(slice_nums)} slice(s) of data...")
    data_path = '/Users/pabil/Downloads/spotify_million_playlist_dataset/data/'
    playlists = load_playlist_slice(data_path, slice_nums=slice_nums)
    if not playlists:
        print("No data loaded. Aborting experiment.")
        return

    all_tracks = [track['track_uri'] for p in playlists for track in p['tracks']]
    unique_tracks = sorted(list(set(all_tracks)))
    track_to_idx = {track: i for i, track in enumerate(unique_tracks)}
    idx_to_track = {i: track for track, i in track_to_idx.items()}
    pid_to_idx = {p['pid']: i for i, p in enumerate(playlists)}

    # Create train-test split
    rows, cols, test_set = [], [], {}
    for p in playlists:
        playlist_idx = pid_to_idx[p['pid']]
        tracks = [t['track_uri'] for t in p['tracks']]
        if len(tracks) > 5:
            np.random.shuffle(tracks)
            num_holdout = int(len(tracks) * 0.2)
            test_set[playlist_idx] = [uri for uri in tracks[-num_holdout:] if uri in track_to_idx]
            train_tracks = tracks[:-num_holdout]
        else:
            train_tracks = tracks
        for track_uri in train_tracks:
            if track_uri in track_to_idx:
                rows.append(playlist_idx)
                cols.append(track_to_idx[track_uri])

    interaction_matrix_train = csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(len(playlists), len(unique_tracks)))

    # --- 2. Model Training ---
    print(f"Training SVD model with n_components={n_components}...")
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    playlist_embeddings = svd.fit_transform(interaction_matrix_train)
    track_embeddings = svd.components_.T

    # --- 3. Evaluation ---
    print(f"Evaluating model with k={k}...")
    knn = NearestNeighbors(n_neighbors=k*2, metric='cosine').fit(track_embeddings)
    avg_precision = 0
    avg_recall = 0
    eval_count = 0

    for user_idx, holdout_items in tqdm(test_set.items(), desc="Evaluating"):
        input_track_indices = interaction_matrix_train[user_idx].indices
        if len(input_track_indices) == 0 or len(holdout_items) == 0:
            continue

        playlist_vector = np.mean(track_embeddings[input_track_indices], axis=0).reshape(1, -1)
        distances, indices = knn.kneighbors(playlist_vector, n_neighbors=k + len(input_track_indices))
        
        recommendations = []
        train_uris = {idx_to_track[i] for i in input_track_indices}
        for idx in indices.flatten():
            rec_uri = idx_to_track[idx]
            if rec_uri not in train_uris:
                recommendations.append(rec_uri)
        
        avg_precision += precision_at_k(k, recommendations[:k], holdout_items)
        avg_recall += recall_at_k(k, recommendations[:k], holdout_items)
        eval_count += 1

    avg_precision /= eval_count
    avg_recall /= eval_count
    
    print(f"Precision@{k}: {avg_precision:.4f}, Recall@{k}: {avg_recall:.4f}")

    # --- 4. Store Results ---
    experiment_results.append({
        'slice_nums': slice_nums,
        'data_size': len(playlists),
        'n_components': n_components,
        f'precision_at_{k}': avg_precision,
        f'recall_at_{k}': avg_recall
    })
    print("--- Experiment Complete ---\n")

## 5. Run Experiments

Now you can easily run multiple experiments by calling the `run_experiment` function with different parameters. We'll loop through a few values for `n_components` to see how it affects performance.

In [ ]:
# Clear previous results before starting a new set of experiments
experiment_results = []

# Define the set of n_components values to test
component_options = [60, 61, 62, 63, 64, 65, 66, 67, 68, 69]


# --- Experiment Set 1: Train on 10,000 playlists (slices 0–9) ---
print("--- Starting experiments on n playlists (1 slice is 1k playlists) ---")

slices_for_playlists = list(range(1))  # slices 0–9

# Run experiments for each n_components value
for n_comps in component_options:
    run_experiment(slice_nums=slices_for_playlists, n_components=n_comps, k=10)


# --- Experiment Set 2: Train on 100,000 playlists (slices 0–99) ---
# WARNING: This will take a very long time and require a lot of memory.
## print("\n--- Starting experiments on 100,000 playlists (100 slices) ---")

## slices_for_100k_playlists = list(range(100))  # slices 0–99

## for n_comps in component_options:
    ## run_experiment(slice_nums=slices_for_100k_playlists, n_components=n_comps, k=10)




--- Starting experiments on n playlists (1 slice is 1k playlists) ---
Loading 10 slice(s) of data...
Training SVD model with n_components=50...
Evaluating model with k=10...


Evaluating: 100%|██████████| 9947/9947 [02:42<00:00, 61.19it/s]


Precision@10: 0.0380, Recall@10: 0.0349
--- Experiment Complete ---

Loading 10 slice(s) of data...
Training SVD model with n_components=60...
Evaluating model with k=10...


Evaluating: 100%|██████████| 9947/9947 [03:06<00:00, 53.44it/s]


Precision@10: 0.0391, Recall@10: 0.0368
--- Experiment Complete ---

Loading 10 slice(s) of data...
Training SVD model with n_components=70...
Evaluating model with k=10...


Evaluating: 100%|██████████| 9947/9947 [03:33<00:00, 46.63it/s]


Precision@10: 0.0390, Recall@10: 0.0359
--- Experiment Complete ---

Loading 10 slice(s) of data...
Training SVD model with n_components=80...
Evaluating model with k=10...


Evaluating: 100%|██████████| 9947/9947 [04:02<00:00, 40.97it/s]


Precision@10: 0.0391, Recall@10: 0.0355
--- Experiment Complete ---



# Testing data

## Testing at 5k slices from 50-80 components
- 50 --> Precision@10: 0.0322, Recall@10: 0.0304 
- 60 --> Precision@10: 0.0327, Recall@10: 0.0309 
- 70 --> Precision@10: 0.0300, Recall@10: 0.0293  
- 80 --> Precision@10: 0.0294, Recall@10: 0.0287 

Seems to be degrading between 60-70 

## Testing at 10k slices from 50-80 components
- 50 --> Precision@10: 0.0380, Recall@10: 0.0349
- 60 --> Precision@10: 0.0391, Recall@10: 0.0368
- 70 --> Precision@10: 0.0390, Recall@10: 0.0359 
- 80 --> Precision@10: 0.0391, Recall@10: 0.0355

## 6. Visualize Results

After the experiments are complete, we can convert our results list into a pandas DataFrame and plot the outcome to easily see the impact of our changes.

In [ ]:
if not experiment_results:
    print("No experiment results to plot. Please run some experiments first.")
else:
    results_df = pd.DataFrame(experiment_results)
    print("Experiment Results:")
    print(results_df)
    
    # Plotting Recall vs. n_components
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['n_components'], results_df['recall_at_10'], marker='o')
    plt.title('Model Performance vs. Number of Components')
    plt.xlabel('Number of Components')
    plt.ylabel('Recall@10')
    plt.grid(True)
    plt.xticks(results_df['n_components'])
    plt.show()